# Phase 1 of project: Data Wrangling

- We will be using the mnist dataset for this project.
- The first iteration will use pytorch

In [ ]:
# load the dataset

from keras.datasets import mnist

(train_X, train_y), (test_X, test_y) = mnist.load_data()

print('Train: X=%s, y=%s' % (train_X.shape, train_y.shape))
print('Test: X=%s, y=%s' % (test_X.shape, test_y.shape))

In [ ]:
# plot the dataset

import matplotlib.pyplot as plt

for i in range(9):
    plt.subplot(330 + 1 + i)
    plt.imshow(train_X[i], cmap=plt.get_cmap('gray'))
plt.show()


In [ ]:
import numpy as np
from itertools import product
# pooling the dataset
# mode = max, mean
def pooling_layer2D(X, spatial_extent=2, stride=2, mode='max'):
    X = np.asarray(X)
    h, w = X.shape
    h_out = int((h - spatial_extent) / stride) + 1
    w_out = int((w - spatial_extent) / stride) + 1
    #get the max/mean value in the spatial extent stride by stride
    Y = np.zeros((h_out, w_out))
    for i, j in product(range(h_out), range(w_out)):
        if mode == 'max':
            Y[i, j] = np.max(X[i * stride:i * stride + spatial_extent, j * stride:j * stride + spatial_extent])
        elif mode == 'mean':
            Y[i, j] = np.mean(X[i * stride:i * stride + spatial_extent, j * stride:j * stride + spatial_extent])
    return Y

In [ ]:
import numpy as np

def flatten_layer(X):
    return X.flatten()

In [ ]:
poollayer = pooling_layer2D(train_X[1], spatial_extent=2, stride=2, mode='max')
flattenlayer = flatten_layer(poollayer)

In [ ]:
# create the model

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

model = Sequential()

# Conv2D takes input images and converts them into feature maps
# MaxPooling2D reduces the size of the feature maps, reducing the number of parameters in the network
# Flatten flattens the feature maps into a single vector to pass to the fully connected layers
# Dense is a fully connected layer
# The last layer is a softmax layer with 10 nodes, one for each class (0-9)

model.add(Conv2D(32, (5, 5), input_shape=(28, 28, 1), activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))


In [ ]:
# compile the model

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


In [ ]:
# train the model

from keras.utils import np_utils

train_X = train_X.reshape((train_X.shape[0], 28, 28, 1)).astype('float32')
test_X = test_X.reshape((test_X.shape[0], 28, 28, 1)).astype('float32')

train_X = train_X / 255
test_X = test_X / 255

train_y = np_utils.to_categorical(train_y)
test_y = np_utils.to_categorical(test_y)

model.fit(train_X, train_y, validation_data=(test_X, test_y), epochs=10, batch_size=200, verbose=2)


In [ ]:
# validate the model

scores = model.evaluate(test_X, test_y, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))
